<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Limipieza SOCIOS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.6 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [ ]:
# # CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# CU_SO = pd.read_excel('/content/CU_SO.xlsx')
# TO_SO = pd.read_excel('/content/TO_SO.xlsx')
# AB_SO = pd.read_excel('/content/AB_SO.xlsx')
# GU_SO = pd.read_excel('/content/GU_SO.xlsx')
# CR_SO = pd.read_excel('/content/CR_SO.xlsx')

In [9]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/SO'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['CR_SO.xlsx', 'AB_SO.xlsx', 'TO_SO.xlsx', 'CU_SO.xlsx', 'GU_SO.xlsx']


In [10]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

KeyboardInterrupt: 

In [ ]:
# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

In [ ]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'CODIGO POSTAL': 'CP',
        'MUNICIPIO': 'LOCALIDAD',
        'FECHA DE SOCIO/A': 'FECHA ALTA SOCIO',
        'FECHA INSCRIPCION': 'FECHA ALTA SOCIO',
        'FECHA NAC': 'FECHA NACIMIENTO',
        'F NAC': 'FECHA NACIMIENTO',
        'PROT AUD': 'PROTESIS'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def AB_localidad_CP(df):
  # Extraer el código postal (CP)
  df['CP'] = df['LOCALIDAD'].str.extract(r'(\d+)', expand=False)

  # Extraer el nombre de la localidad (LOCALIDAD)
  df['LOCALIDAD'] = df['LOCALIDAD'].str.extract(r'(\D+)', expand=False).str.strip()
  return df

def CP_year(df):
    if 'FECHA NACIMIENTO' in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df['FECHA NACIMIENTO']):
            df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

        df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
        df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

    if 'CP' in df.columns:
        # Convertir a numérico, dejando como NaN los valores que no sean convertibles
        df['CP'] = pd.to_numeric(df['CP'], errors='coerce').astype('Int64')

        df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
        CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()
        df = df[~CP_loc_filter]

    elif 'LOCALIDAD' in df.columns:
        df = df.dropna(subset=['LOCALIDAD'], how='all')
        loc_filter = df['LOCALIDAD'].isnull()
        df = df[~loc_filter]

    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    return df

def keep_columns(df):
    desired_columns = ['FECHA NACIMIENTO', 'GENERO', 'LOCALIDAD', 'CP', 'PROVINCIA', 'YEAR NACIMIENTO', 'FECHA ALTA SOCIO']
    # Filtrar el DataFrame para incluir solo las columnas que existen en el DataFrame
    filtered_df = df[df.columns.intersection(desired_columns)]
    return filtered_df

In [7]:
for df_key, df in dic_dataframes.items():
    # Apply transformations for DataFrames with keys containing 'AB'
    if 'AB' in df_key:
        dic_dataframes[df_key] = transform_df(df)
        dic_dataframes[df_key] = mapping_names(df)
        dic_dataframes[df_key] = AB_loc(df)
        dic_dataframes[df_key] = kept_columns(df)
    else:
        # Apply different transformations for other DataFrames
        dic_dataframes[df_key] = transform_df(df)
        dic_dataframes[df_key] = mapping_names(df)
        dic_dataframes[df_key] = CP_year(df)
        dic_dataframes[df_key] = kept_columns(df)

NameError: name 'dic_dataframes' is not defined

In [ ]:
# AB_SO = transform_df(AB_SO)
# AB_SO = mapping_names(AB_SO)
# AB_SO = AB_localidad_CP(AB_SO)
# AB_SO = CP_year(AB_SO)
# AB_SO = keep_columns(AB_SO)

# CU_SO = transform_df(CU_SO)
# CU_SO = mapping_names(CU_SO)
# CU_SO = CP_year(CU_SO)
# CU_SO = keep_columns(CU_SO)

# CR_SO = transform_df(CR_SO)
# CR_SO = mapping_names(CR_SO)
# CR_SO = CP_year(CR_SO)
# CR_SO = keep_columns(CR_SO)

# GU_SO = transform_df(GU_SO)
# GU_SO = mapping_names(GU_SO)
# GU_SO = CP_year(GU_SO)
# GU_SO = keep_columns(GU_SO)

# TO_SO = transform_df(TO_SO)
# TO_SO = mapping_names(TO_SO)
# TO_SO = CP_year(TO_SO)
# TO_SO = keep_columns(TO_SO)

In [ ]:
CLM_SO = pd.concat([AB_SO, CU_SO, CR_SO, GU_SO, TO_SO])

In [ ]:
CU_SO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   GÉNERO                           61 non-null     object        
 1   APELLIDOS                        60 non-null     object        
 2   USUARIO/COLABORADOR/PADRE/MADRE  61 non-null     object        
 3   DIRECCIÓN                        61 non-null     object        
 4   CP                               60 non-null     float64       
 5   MUNICIPIO                        61 non-null     object        
 6   PROVINCIA                        61 non-null     object        
 7   FECHA DE SOCIO/A                 58 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 3.9+ KB


In [ ]:
CR_SO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id socio            120 non-null    object
 1   Nombre y Apellidos  121 non-null    object
 2   GENERO              120 non-null    object
 3   LOCALIDAD           118 non-null    object
 4   Código postal       117 non-null    object
dtypes: object(5)
memory usage: 4.9+ KB


In [ ]:
columna =
filas_nulas = CR_SO[CR_SO['YEAR NACIMIENTO'].isnull()]
filas_nulas

KeyError: 'YEAR NACIMIENTO'

In [ ]:
# Save DataFrame to an Excel file
excel_file = 'CU_SO_limpio.xlsx'
CU_SO.to_excel(excel_file, index=False)

excel_file = 'GU_SO_limpio.xlsx'
GU_SO.to_excel(excel_file, index=False)

excel_file = 'CR_SO_limpio.xlsx'
CR_SO.to_excel(excel_file, index=False)

excel_file = 'TO_SO_limpio.xlsx'
TO_SO.to_excel(excel_file, index=False)

excel_file = 'AB_SO_limpio.xlsx'
AB_SO.to_excel(excel_file, index=False)

excel_file = 'CLM_SO.xlsx'
CLM_SO.to_excel(excel_file, index=False)